<a href="https://colab.research.google.com/github/Stalin124/TesisScriptdeEntrenamiento/blob/main/EntrenamientoYOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Paso 1: Instalación y Configuración Inicial
!pip install ultralytics roboflow --quiet

from roboflow import Roboflow
from ultralytics import YOLO
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import shutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/12

In [ ]:
# ✅ Paso 2: Importación de dataset desde Roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="OGHKOE8ss7hxmeOOSa6X")
project = rf.workspace("clasificacin-vehcular").project("proyectotesisvalido-general")
version = project.version(2)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ProyectoTesisValido-GENERAL-2 in yolov11:: 100%|██████████| 3648/3648 [00:02<00:00, 1687.40it/s]


In [ ]:
# ✅ Paso 3: Aplicación de oversampling a clases minoritarias
# Paso 3.1: Se definen las clases minoritarias
minority_classes = ["BICICLETA", "BUS", "PESADO", "MOTOCICLETA","TAXI"]  # Múltiples clases
n_copias = 5  # Número de veces que se duplicarán las imágenes

# Paso 3.2: Duplicado de imágenes y etiquetas
import os, shutil

images_dir = os.path.join(dataset.location, "train/images")
labels_dir = os.path.join(dataset.location, "train/labels")

for clase in minority_classes:
    for i in range(n_copias):
        for filename in os.listdir(images_dir):
            if clase in filename:
                src_img = os.path.join(images_dir, filename)
                dst_img = os.path.join(images_dir, f"copy{i}_{clase}_{filename}")
                shutil.copy(src_img, dst_img)

                # Verificación de etiquetado correspondiente
                label_file = filename.replace(".jpg", ".txt").replace(".png", ".txt")
                src_lbl = os.path.join(labels_dir, label_file)
                dst_lbl = os.path.join(labels_dir, f"copy{i}_{clase}_{label_file}")
                if os.path.exists(src_lbl):
                    shutil.copy(src_lbl, dst_lbl)

In [ ]:
# ✅ Paso 4: Entrenamiento del modelo YOLOv11
model = YOLO("yolo11n.pt")  # Versión de Trabajo yolov11n.pt

results = model.train(
    data=os.path.join(dataset.location, "data.yaml"),
    epochs=50,
    imgsz=960,
    batch=8,
    name="roboflow_yolov11_model",
    device=0 if torch.cuda.is_available() else 'cpu',
    patience=20,
    verbose=True
)

100%|██████████| 5.35M/5.35M [00:00<00:00, 156MB/s]


Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/ProyectoTesisValido-GENERAL-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=roboflow_yolov11_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, p

100%|██████████| 755k/755k [00:00<00:00, 36.5MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,591,010 parameters, 2,590,994 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1462.9±905.4 MB/s, size: 272.5 KB)


...

      50/50      3.61G     0.7879     0.4145     0.9118         50        960: 100%|██████████| 152/152 [00:53<00:00,  2.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:07<00:00,  3.29it/s]

                   all        403       2528      0.929      0.835      0.907      0.696



50 epochs completed in 0.893 hours.
Optimizer stripped from runs/detect/roboflow_yolov11_model/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/roboflow_yolov11_model/weights/best.pt, 5.5MB

Validating runs/detect/roboflow_yolov11_model/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:09<00:00,  2.69it/s]


                   all        403       2528      0.909      0.857      0.905        0.7
             BICICLETA         12         13      0.885      0.769      0.822      0.593
                   BUS         37         37      0.973      0.959      0.974      0.843
           MOTOCICLETA        163        220      0.911      0.791      0.892      0.561
            PARTICULAR        388       1486      0.926      0.926      0.953      0.764
                PESADO         72         78      0.835      0.744      0.826      0.677
                  TAXI        330        694      0.924      0.952      0.965      0.762
Speed: 0.7ms preprocess, 4.5ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs/detect/roboflow_yolov11_model
